# ECE 884 Deep Learning - Final Project
*Notebook 1 of 3*<br>
**Name:** Syed Kashif Mujtaba Kamoonpuri<br>
**Email:** kamoonpu@msu.edu

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import datetime

**This notebook contains the following dataframe:**<br>
- cases: Time-series data containing number of confirmed cases of COVID-19 globally
- cases_us: Time-series data containing number of confirmed cases of COVID-19 in US
- deaths: Time-series data containing number of deaths dut to COVID-19 globally
- deaths_us: Time-series data containing number of deaths dut to COVID-19 in US

**Reading CSV files**

In [2]:
cases = pd.read_csv('covid19_confirmed_global.csv')
print(cases.shape)

(266, 206)


In [3]:
cases_us = pd.read_csv('covid19_confirmed_US.csv')
print(cases_us.shape)

(3340, 214)


In [4]:
deaths = pd.read_csv('covid19_deaths_global.csv')
print(deaths.shape)

(266, 206)


In [5]:
deaths_us = pd.read_csv('covid19_deaths_US.csv')
print(deaths_us.shape)

(3340, 215)


In [6]:
cases_us.iloc[:,:15].head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,1/22/20,1/23/20,1/24/20,1/25/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",0,0,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",0,0,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",0,0,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",0,0,0,0


**Removing redundant columns in cases_us and deaths_us**

In [7]:
col_drop = ['UID','iso2','iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','8/11/20']
to_keep = []
for col in cases_us.columns:
    if col not in col_drop:
        to_keep.append(col)

**Grouping cases_us and deaths_us by Province_State and Country_Region**

In [8]:
cases_us = cases_us.loc[:, to_keep]
cases_us = cases_us.groupby(['Province_State', 'Country_Region']).sum()
cases_us.reset_index(inplace=True)

deaths_us = deaths_us.loc[:, to_keep]
deaths_us = deaths_us.groupby(['Province_State', 'Country_Region']).sum()
deaths_us.reset_index(inplace=True)

print(cases_us.shape)
print(deaths_us.shape)

(58, 204)
(58, 204)


**Removing redundant rows & columns in cases dataframes**

In [9]:
col_list = list(cases.columns)
col_list[0] = 'Province_State'
col_list[1] = 'Country_Region'
cases.columns = col_list

col_list.pop(col_list.index('Lat'))
col_list.pop(col_list.index('Long'))

cases = cases.loc[:, col_list]
cases = cases.loc[cases.Country_Region != 'US', :]
cases.head()

,Province_State,Country_Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,36710,36710,36747,36782,36829,36896,37015,37054,37054,37162
1,NaN,Albania,0,0,0,0,0,0,0,0,...,5396,5519,5620,5750,5889,6016,6151,6275,6411,6536
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,30950,31465,31972,32504,33055,33626,34155,34693,35160,35712
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,925,925,937,939,939,944,955,955,955,963
4,NaN,Angola,0,0,0,0,0,0,0,0,...,1164,1199,1280,1344,1395,1483,1538,1572,1672,1679


### Converting DataFrames from wide format to long format

In [10]:
cases = pd.melt(cases,id_vars=['Province_State', 'Country_Region'],var_name='Date', value_name='ConfirmedCases')
cases['Date'] = pd.to_datetime(cases['Date'])
cases.head()

,Province_State,Country_Region,Date,ConfirmedCases
0,NaN,Afghanistan,2020-01-22,0
1,NaN,Albania,2020-01-22,0
2,NaN,Algeria,2020-01-22,0
3,NaN,Andorra,2020-01-22,0
4,NaN,Angola,2020-01-22,0


In [11]:
cases_us = pd.melt(cases_us,id_vars=['Province_State', 'Country_Region'],var_name='Date', value_name='ConfirmedCases')
cases_us['Date'] = pd.to_datetime(cases_us['Date'])
cases_us.head()

,Province_State,Country_Region,Date,ConfirmedCases
0,Alabama,US,2020-01-22,0
1,Alaska,US,2020-01-22,0
2,American Samoa,US,2020-01-22,0
3,Arizona,US,2020-01-22,0
4,Arkansas,US,2020-01-22,0


**Concatenating cases and cases_us into total_cases**

In [12]:
total_cases = pd.concat([cases, cases_us])
total_cases.shape

(65246, 4)

**Removing redundant rows & columns in deaths dataframes**

In [13]:
col_list = list(deaths.columns)
col_list[0] = 'Province_State'
col_list[1] = 'Country_Region'
deaths.columns = col_list

col_list.pop(col_list.index('Lat'))
col_list.pop(col_list.index('Long'))

deaths = deaths.loc[:, col_list]
deaths = deaths.loc[deaths.Country_Region != 'US', :]
deaths.head()

,Province_State,Country_Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,1283,1284,1288,1288,1294,1298,1307,1312,1312,1328
1,NaN,Albania,0,0,0,0,0,0,0,0,...,161,166,172,176,182,188,189,193,199,200
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,1223,1231,1239,1248,1261,1273,1282,1293,1302,1312
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,52,52,52,52,52,52,52,52,52,52
4,NaN,Angola,0,0,0,0,0,0,0,0,...,54,55,58,59,62,64,67,70,75,78


In [14]:
deaths = pd.melt(deaths,id_vars=['Province_State', 'Country_Region'],var_name='Date', value_name='Fatalities')
deaths['Date'] = pd.to_datetime(deaths['Date'])
deaths.head()

,Province_State,Country_Region,Date,Fatalities
0,NaN,Afghanistan,2020-01-22,0
1,NaN,Albania,2020-01-22,0
2,NaN,Algeria,2020-01-22,0
3,NaN,Andorra,2020-01-22,0
4,NaN,Angola,2020-01-22,0


In [15]:
deaths_us = pd.melt(deaths_us,id_vars=['Province_State', 'Country_Region'],var_name='Date', value_name='Fatalities')
deaths_us['Date'] = pd.to_datetime(deaths_us['Date'])
deaths_us.head()

,Province_State,Country_Region,Date,Fatalities
0,Alabama,US,2020-01-22,0
1,Alaska,US,2020-01-22,0
2,American Samoa,US,2020-01-22,0
3,Arizona,US,2020-01-22,0
4,Arkansas,US,2020-01-22,0


**Concatenating deaths and deaths_us into total_cases**

In [16]:
total_deaths = pd.concat([deaths, deaths_us])
total_deaths.shape

(65246, 4)

**Joining total_cases and toal_deaths**

In [17]:
total_deaths.set_index(['Province_State','Country_Region', 'Date'], inplace=True)
data = pd.merge(total_cases,total_deaths, on=['Province_State','Country_Region','Date'], how='left')
data.sort_values(by=['Country_Region', 'Province_State', 'Date'], inplace=True)
data.reset_index(inplace=True, drop=True)
data.head()

,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,NaN,Afghanistan,2020-01-22,0,0
1,NaN,Afghanistan,2020-01-23,0,0
2,NaN,Afghanistan,2020-01-24,0,0
3,NaN,Afghanistan,2020-01-25,0,0
4,NaN,Afghanistan,2020-01-26,0,0


**Checking for null values**

In [18]:
data.isnull().sum()

Province_State    37168
Country_Region        0
Date                  0
ConfirmedCases        0
Fatalities            0
dtype: int64

**Joining Country_Region and Province_State into 'Region' to get rid of missing values**

In [19]:
data['Region'] = np.nan
data.loc[data.Province_State.isnull(), 'Region'] = data.loc[data.Province_State.isnull(), 'Country_Region']
data.loc[data.Province_State.notnull(), 'Region'] = data.loc[data.Province_State.notnull(), 'Country_Region']\
                                + ' ' + data.loc[data.Province_State.notnull(),'Province_State'].astype('str')
data = data.loc[:, ['Date', 'Region', 'ConfirmedCases', 'Fatalities']]
data.loc[500:1000,:]

,Date,Region,ConfirmedCases,Fatalities
500,2020-04-27,Algeria,3517,432
501,2020-04-28,Algeria,3649,437
502,2020-04-29,Algeria,3848,444
503,2020-04-30,Algeria,4006,450
504,2020-05-01,Algeria,4154,453
...,...,...,...,...
996,2020-07-28,Angola,1000,47
997,2020-07-29,Angola,1078,48
998,2020-07-30,Angola,1109,51
999,2020-07-31,Angola,1148,52


**Checking for null values**

In [20]:
data.isnull().sum()

Date              0
Region            0
ConfirmedCases    0
Fatalities        0
dtype: int64

**Saving as CSV**

In [21]:
data.to_csv('covid19_CF.csv', index=False)